# Standard Genomics File Formats
------
### Learning Objectives:

+ Learn the standard file formats of three common genomics file types
    + GFF files for storing genomic annotation data
    + Fasta files for storing assembled sequence data
    + Fastq files for storing raw read data

+ Learn basic BASH coding to explore file contents

+ Learn to build complex BASH code  by combining basic BASH commands



## Genomic Annotation Data, GFF/GTF file Format
------
The standard genomic annotation format is called a GFF/GTF file, this is a tab delimited (tsv) file that indicates the positions of genomic features of interest in a genome. 

Features of interest could be genes, transcripts, non-coding RNA, tRNA, rRNA, 3'-UTRs, and more. When building a GFF file you have the option to select the features annotated. Most files include at least genes, transcripts, tRNA, and rRNA. 

<p align="center">
  <img src="images/gtf.png" width="100%"/>
</p>



<table>
<tr><th>GFF files all have 9 tab delimited fields, as shown in the figure above. Each field is described briefly below.</th></tr>
<tr><td><table></table>


|Column number | Column name | Column description|
|--- | --- | ---|
|Column 1 | Seqname | The sequence ID being annotated, usually the chromosome or scaffold name|
|Column 2 | Source | The source of the annotations, usually the software used to annotate features|
|Column 3 | Feature type | The type of feature annotated|
|Column 4 | Start | The starting coordinate of the feature|
|Column 5 | End | The end coordinate of the feature|
|Column 6 | Score | The score of the feature, this could be an e-value or p-value depending on the annotation software|
|Column 7 | Strand | + for positive strand (relative to the landmark), - for minus strand, and . for features that are not stranded|
|Column 8 | Frame | For features of type "CDS", the frame can be 0, 1, or 2, indicating the number of bases forward from the start of the current CDS feature the next codon begins|
|Column 9 | Feature attributes | A list of feature attributes in the format tag=value. Multiple tag=value pairs are separated by semicolons|

</td></tr> </table>


There is a gff format file in the `gcp_research_workflow` directory (downloaded in lesson 2) that is an annotation file for the Sars-CoV2 genome called `gcp_research_workflow/sequence.gff3`. let's have a look at this file to familiarize ourselves with the file format and some ways to work with this format in the terminal environment. 

#### wc

To start we will use the *word count* `wc` command with the flag `-l` to indicate we would like to know how many lines are in the file. As you can see from the example figure there will be one line for each annotated feature.

In [ ]:
%%bash

# let's count the lines in the file
# We will use the "wc" command with the "-l" flag to indicate we want to count the number of lines in the file
wc -l gcp_research_workflow/sequence.gff3

#### cut

Sars-CoV2 is a virus, and thus has a pretty small genome and a limited number of features. Now we will use the `cut` command to see which features are annotated in this file. By default `cut` will use the tab to delimit columns and the flag `-f` can be used to select a column of interest to print to the screen. Remember the type of feature is listed in the third column, so we will use `cut -f3` to pull the information from the third column.

In [ ]:
%%bash

# Now let's isolate the information in the 3rd column to check what type of features are annotated in this genome
# We will use the "cut" command with the "-f" flag to pull out the thrid column
cut -f 3 gcp_research_workflow/sequence.gff3


Most of the annotated features are coding sequences with some non-coding features annotated. If we were interested in which features were annotated in this file the information is here, but it is poorly organized with this command. We can build on the command we used above using the *pipe* `|` which takes the output from the command on the left of the pipe and sends it as input to the command on the right of the pipe. 

#### sort

Here we can combine the `cut -f3` command we used earlier with a `sort` command using the flag `-u` to report each unique value only once and produce a concise list of features annotated in the file. 


In [ ]:
%%bash

# That is a long list, let's use the "|" which enables us to link commands where the output of the command on the left of the pipe becomes the input for the command on the right of the pipe
# We will combine our earlier command with the "sort" command and the "-u" flag to reduce the list to one line for each unique entry in the column
cut -f 3 gcp_research_workflow/sequence.gff3|sort -u


The output of this command provides a more clear and organized list of the types of features annotated in this file. You can see there are seven types of features annotated in this file. 

#### grep

Next let's pull out the annotation lines that correspond to coding sequences, feature type *CDS*, with the command `grep`. The `grep` command is a powerful tool for searching for text strings in a file. The syntax of the `grep` command is `grep PATTERN FILENAME`.

In [ ]:
%%bash

# Use the grep command to isolate CDS features
grep "CDS" gcp_research_workflow/sequence.gff3


There are quite a few lines there, and a lot of information on each line. let's combine the `grep` command with a `cut` command to pull out the genome name, start position, and stop position for each coding sequence in this file. Remember the genome name is the first field, the start position is the fourth field, and the stop position is the fifth field. 

In [ ]:
%%bash

# Lastly we will save the output to a new text file with the redirect command ">" 
grep "CDS" gcp_research_workflow/sequence.gff3|cut -f1,4,5 > ref_locations.txt

# Take a look at the first couple lines of the output file
head -n3 ref_locations.txt

The output in this format is certainly less noisy but could use some more annotation. 

Let's add the annotation information which is part of the feature attributes column, the ninth column in this file. This column contains a lot of information and each attribute is in the format *tag=value* and separated from the next attribute by a semi-colon. Some attributes are missing in some of the coding sequence entries, meaning there are a different number of `;` on each line and we won't be able to simply use the `cut` command as we did with the position information. However, each line has a *product* annotation as the second to last attribute. We can build a fairly complex command with `|` to isolate the annotation for each coding sequence. Here are the steps of the command in the video below:

<span style="color:black">Learn more about piping commands in [this video](https://www.youtube.com/watch?v=J5dmCAYOh9Q&list=PLXaEJPtnQ4w7Vu7vqWbttBjUGrPp4Qa7b&index=4)</span>

1. `grep "CDS" gcp_research_workflow/sequence.gff3` pull the lines that have the pattern "CDS"
2. `cut -f 9` isolate the ninth field of the output from step 1
3. `rev` reverse the output from step 2 so the last letter is first, etc.
4. `cut -d ";" -f 2` cut using the ";" as the delimiter instead of the tab and take the second field
5. `rev` reverse the string again so that the string is readable
6. `cut -d "=" -f 2` cut using the "=" as the delimited and take the second field
7. `>products.txt` save the output to the file products.txt

Use the coding fields below so you can see the output of each step as we build the complexity of the command. Each of these commands (except the last one) ends in the `head -n1` command to limit the output to 1 line and improve the readability of the output.

In [ ]:
%%bash

# Step 1
grep "CDS" gcp_research_workflow/sequence.gff3|head -n1

In [ ]:
%%bash

# Step 2
grep "CDS" gcp_research_workflow/sequence.gff3|cut -f9|head -n1

In [ ]:
%%bash

# Step 3
grep "CDS" gcp_research_workflow/sequence.gff3|cut -f9|rev|head -n1

In [ ]:
%%bash

# Step 4
grep "CDS" gcp_research_workflow/sequence.gff3|cut -f9|rev|cut -d ";" -f2|head -n1


In [ ]:
%%bash

# Step 5
grep "CDS" gcp_research_workflow/sequence.gff3|cut -f9|rev|cut -d ";" -f2|rev |head -n1

In [ ]:
%%bash

# Step 6
grep "CDS" gcp_research_workflow/sequence.gff3|cut -f9|rev|cut -d ";" -f2|rev|cut -d "=" -f2|head -n1


In [ ]:
%%bash

# Putting it all together and saving it to a file
grep "CDS" gcp_research_workflow/sequence.gff3|cut -f9|rev|cut -d ";" -f2|rev|cut -d "=" -f2> products.txt

In [ ]:
%%bash

head products.txt


You can see how using the pipe `|` enables you build pretty complex commands by repeating a series of simpler commands on the output of previous commands. Now let's add our annotations to our location information to create a file with the positions and annotation information for all coding sequences from the Sars-CoV2 genome. 

For this we will use the `paste` command, this command paste the contents of files together. By default the contents of each file will be pasted together with a tab, however as with `cut` this can be modified with the `-d` flag. Here we will use the default to paste together the information from CDS annotation lines. 

In [ ]:
%%bash

# Lastly let's combine the files we made with the "paste" command
paste ref_locations.txt products.txt > cds_info.txt

# let's look at our new file 
head cds_info.txt

# Much easier to read!!! 

## Genomic Sequence Data, Fasta Files
----------------

Fasta files are used to store various types of 'long form' genomic data. Fasta files most often contain genomic sequences in either nucleotide or amino acid data, but the fasta file format can also be used to store phenotypic data of a given organism.

Regardless of the sequence alphabet the format of a fasta file is defined by two types of lines:
1. A header line begins with the `>` character and has a description of the sequence on the lines that follow
2. A sequence line contains the characters of the sequence described by the header line

<p align="center">
  <img src="images/fasta.png" width="70%"/>
</p>


--------------

<p align="center">
  <img src="images/fasta1.png" width="70%"/>
</p>



#### Decoding the Contents of a Fasta File

Fasta files containing sequence data can be of many different types. Whole genome sequences, assembled scaffolds from reads, aligned protein sequences, or a collection of coding sequences from an organisms are several examples of the types of data you can find in a fasta file. 

Hopefully not often, but from time to time you may come across a fasta file you would like to work with, but are unsure of the contents of the file. There are a couple of features of various data types that you can use to determine the contents of the file. 

There are almost always more sequence lines than header lines in a fasta file, but data like annotated coding sequences and sequence alignments will have more header lines than files containing assembled scaffolds or complete genome data. In most complete genome fasta files and very good assembled scaffolds you will have only 1 or 2 header lines with many lines of sequence data. 

In the examples above you can see that at times the header doesn't contain much information as in the example at the top of the figure. However, the example at the bottom of the figure provides an accession number, a description of the protein, and the species the sequence came from. This second example is a very good example of what information is helpful to include in a sequence header. 

One last feature is the alphabet of letters in the sequence data. As I mentioned earlier there are rare cases where a fasta file encodes phenotypic characters and in those cases you will see mostly binary characters 1 and 0 in the sequence lines, but this is rare. Most sequences will be either nucleotide sequences or protein sequences, and these are pretty easily distinguished by their very different alphabets. The sequences in the top half of the figure above are nucleotide data in the fasta file  format, this is clear from the limited sequence alphabet (*ATCG*). Sequences in the bottom half of the figure are in the protein format, indicated by the expanded sequence alphabet (*ACDEFGHIKLMNPQRSTVWY*). 


I have added two unlabeled fasta files with sequence data to the directory `gcp_research_workflow`. let's look at some of the features we outlined above to determine the contents of each file, and then rename the files so that we have a better system for knowing what each file contains. 

Start by determining how many lines are in the file with the word count `wc` command using the flag `-l`.




In [ ]:
%%bash

# List the number of lines in the first fasta file with the word count (wc) command 
# Using the flag "-l" indicate you want to know how many lines (rather than words) are in the file

wc -l gcp_research_workflow/sequence-1.fasta


Next we want to know how many of the total lines in the file are header lines beginning with the `>` symbol. For this we will use the search command `grep`. As you saw in the sections above, this command is useful for isolating interesting information from a large file. We can extend the utility of this command even further by combining it with a regular expression. 


#### Regular Expressions

Regular expressions are symbols in coding languages that stand for a pattern. The meaning of many regular expressions are preserved across multiple coding languages because they are useful in so many aspects of coding. 


<table>
<tr><th>Commonly used regular expressions.</th></tr>
<tr><td><table></table>
    
|Operator | Effect|
|---|---|
|\* | wildcard stands for any number of anything|
|^ | start of the line|
|$ | end of the line|
|[0-9] or \d| any number (0123456789)|
|[a-z]| any lowercase letter|
|[A-Z]| any uppercase letter|
|\t | a tab|

</td></tr> </table>


To count the number of times a feature was annotated in a GFF file as either the 5'-UTR or the 3'-UTR I could use the following command:

`grep -c "[0-9]'-UTR" sequence.gff`

The `-c` flag would return a count of the number of times the patterns 0'-UTR, 1'-UTR, 2'-UTR, 3'-UTR, 4'-UTR, 5'UTR, 6'-UTR, 7'-UTR, 8'-UTR, and 9'-UTR were found in the file. Of course only 5'-UTR and 3'-UTR would be found but the regular expression is looking for any number before the rest of the pattern provided to the `grep` command.

Here we can use the regular expression `^` with the `grep -c` command to count lines where the pattern `>` is found at the start of the line.


In [ ]:
%%bash

# Check how many lines start with the ">" character in the first file
# The grep command will print out the lines that match the pattern we are looking for
# Notice that the pattern I'm looking for (in quotes) uses the regular expression `^` to indicate that I want lines that begin with the ">" 
# Using the `-c` flag with the grep command will count the number of lines that match that pattern
grep -c "^>" gcp_research_workflow/sequence-1.fasta

In this case about half of the lines of this file are header lines, that is they begin with `>`, and the other half are sequence lines. It is unlikely with this ratio of sequence to header lines that this file contains a whole genome or an assembly scaffold.

Next, we can check to see if there is useful information in the header lines. Here we can use the `grep` command again with the regular expression to isolate the header lines, but this time for ease of interpretation we will limit the matches returned by grep with the `-m` flag.

In [ ]:
%%bash

# Using the `-m 3` flag with the grep command will only return the first 3 matches (or whatever number you type after the flag)
grep -m 3 "^>" gcp_research_workflow/sequence-1.fasta

#### head, tail, cat, & more

Here we can see that the header lines indicate the species the sequences come from, and there are three different species listed, but there isn't any description of the sequence content in the header beyond that. let's look at some of the sequence data with the `head` and `tail` commands to determine what type of data is in this file. 

We could look at the contents of the files with the `cat` command, which prints all lines of the file to the screen. For small files this is okay, but for larger files like fasta files this amount of output can be overwhelming and not very useful. 

One more way to view the contents of a file is the `more` command which displays data one "screenfull" of lines at a time (the number of lines depends on the size of your screen and the font size). This command is interactive so it won't work in the Jupyter notebook, but is very useful for scrolling through data in the terminal window. The `space-bar` key will 'scroll' one line at a time and the `return` key will 'scroll' one screen full  of lines at a time. If you would like to stop scrolling and return to your prompt use the `ctrl + C` keys. The `ctrl +C` is a universal signal to the terminal that you would like to abort the command that is currently running.

In [ ]:
%%bash

# The head command by default shows only the first ten lines of the file
# You can adjust the number of lines shown with the flag `-n 6` for the first 6 lines
head -n 6 gcp_research_workflow/sequence-1.fasta

In [ ]:
%%bash

# The tail command by default shows the last ten lines of the file 
# You can be adjust the number of lines shown with the `-n 20` flag to show the last 20 lines of the file
tail -n 20 gcp_research_workflow/sequence-1.fasta

The sequence alphabet of this file indicates that these are protein sequences and you'll notice that each sequences is about the same length. There isn't a lot of information in the headers, but we can see that each sequence comes from a different species and we know that there are the same number of headers and sequence lines. Together this information indicates that these sequences are from a protein alignment. To be sure we can use NCBI's [BLAST tool](https://blast.ncbi.nlm.nih.gov/Blast.cgi?PROGRAM=blastp&PAGE_TYPE=BlastSearch&LINK_LOC=blasthome) to check the identity of some of these sequences.  

To save you the trouble I will tell you this file is a multi-sequence alignment of the S11 ribosomal protein from various proteobacteria. Now that we know what this file is, let's copy it into a new directory  with the command `cp`. The syntax of this command is similar to the copy command we used to copy the files from the Google bucket to your Jupyter notebook, `cp SOURCE DESTINATION` as before we will replace the arguments with paths.

In [ ]:
%%bash

# Make a directory to store annotated sequence data
mkdir new_dir

# Copy the file "sequence-1.fasta" into the directory you made called new_dir
cp gcp_research_workflow/sequence-1.fasta new_dir/

# Check to see that the file was copied
ls new_dir

#### mv

Now let's rename the file to something that is more descriptive like `S11_proteobacteria_proteins.fasta`. We can rename files with the `mv` command. This is flexible in that the activity of the command is dictated by the types of arguments provided. The command can be used to move files by using the command syntax `mv SOURCE DESTINATION` or it can be used to rename files by using the syntax `mv CURRENTFILENAME NEWFILENAME`. Here we provide 2 filenames as arguments, so the command will rename the first file with the second argument.

In [ ]:
%%bash

# Rename the file with mv command
mv new_dir/sequence-1.fasta new_dir/S11_proteobacteria_proteins.fasta

# Check to see that the name has changed
ls new_dir

In [ ]:
%%bash

# Move the sequence-1 file from gcp_research_workflow to new_dir
mv gcp_research_workflow/sequence-1.fasta new_dir/

# Check to see that the file has moved
ls new_dir

<div class="alert alert-block alert-warning">
    <i class="fa fa-question-circle-o" aria-hidden="true"></i>
    <b>TEST YOUR SKILLS</b> 
      <p>Practice your skills in the code block below</p>
    <div style="background-color: white ; color:black; padding: 3px;"> 1. List the number of lines in each fasta file with the word count (wc) command <br> 2. Using the flag "-l" indicate you want to know how many lines (rather than words) are in the file <br>3. Using the `-c` flag with the grep command will count the number of lines that match that pattern <br>4. Using the `-m 3` flag with the grep command will only return the first 3 matches (or whatever number you type after the flag)<br>5. The head command by default shows only the first ten lines of the file but you can adjust the number of lines shown with the flag `-n 100` for the first 100 lines.<br><br> Run the #FLASHCARD code block to see the answer.</div>
    
</div>
 

In [ ]:
%%bash

# TEST YOUR SKILLS (enter and run your answers here)

# List the number of lines in each fasta file with the word count (wc) command 

# Using the flag "-l" indicate you want to know how many lines (rather than words) are in the file

# Using the `-c` flag with the grep command will count the number of lines that match that pattern

# Using the `-m 3` flag with the grep command will only return the first 3 matches (or whatever number you type after the flag)

# The head command by default shows only the first ten lines of the file but you can adjust the number of lines shown with the flag `-n 100` for the first 100 lines



In [ ]:
# FLASHCARD
from IPython.display import IFrame
IFrame("quiz_files/quiz3-1.html", width=600, height=250)

## Raw Sequencing Data, FASTQ Files
------------------------------

FASTQ files are arguably the workhorse file format of bioinformatics. FASTQs are used to store sequence reads generated in next-generation sequencing (NGS) experiments. Similarly to FASTA files, FASTQ files contain a header line, followed by the sequence read, however individual quality scores of base calls from the sequencer are included for each record in a FASTQ file.

Here is what a the first record of an example FASTQ file looks like
```
@SRR1039508.1 HWI-ST177:290:C0TECACXX:1:1101:1225:2130 length=63
CATTGCTGATACCAANNNNNNNNGCATTCCTCAAGGTCTTCCTCCTTCCCTTACGGAATTACA
+
HJJJJJJJJJJJJJJ########00?GHIJJJJJJJIJJJJJJJJJJJJJJJJJHHHFFFFFD
```

**Four rows exist for each record in a FASTQ file:**
- **Row 1:** Header line that stores information about the read (always starts with an `@`), such as the *instrument ID*, *flowcell ID*, *lane on flowcell*, *file number*, *cluster coordinates*, *sample barcode*, etc.
- **Row 2:** The sequence of bases called
- **Row 3:** Usually just a `+` and sometimes followed by the read information in line 1
- **Row 4:** Individual base qualities (must be same length as line 2)

Quality scores, also known as **Phred scores**, in row 4 represent the probability that the associated base call is incorrect. A base with a Phred score of `10` has a `1 in 10` chance of being an incorrectly called base, or *90%*, as in the table below:

Phred Score | Probability of incorrect call | Accuracy
-------|-------|-------
10 | 1 in 10 | 90%
20 | 1 in 100 | 99%
30 | 1 in 1000 | 99.9%
40 | 1 in 10,000 | 99.99%
50 | 1 in 100,000 | 99.999%
60 | 1 in 1,000,000 | 99.9999%


However, we can clearly see that the fourth line does not list probabilities. Instead, quality scores are encoded by a character that is associated with an **ASCII (American Standard Code for Information Interchange)** characters. ASCII codes provide a convenient way of representing a number with a character.

In FASTQ files, Q-score is linked to a specific ASCII character by **adding 33 to the Phred-score**, and matching the resulting number with its *ASCII* character according to the standard code.

| Symbol | ASCII Code | Q-Score |
|--------|------------|---------|
| !      | 33         | 0       |
| "      | 34         | 1       |
| #      | 35         | 2       |
| $      | 36         | 3       |
| …      | …          | …       |
| ?      | 63         | 30      |
| @      | 64         | 31      |
| A      | 65         | 32      |
| …      | …          | …       |

 You can see the full table [here](https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/QualityScoreEncoding_swBS.htm). This encoding means that quality scores only take up **1 byte per value** in the FASTQ file (saves disk space).

For example, the first base call in our sequence example above, the `C` has a quality score encoded by an `H`, which corresponds to a Q-score of 34, meaning this is a good quality base call.


In contrast, you can see a stretch of `#`s indicating a Q-score of 2. Looking at the FASTQ record (line 2), these correspond to a string of `N` calls, which are bases that the sequencer was not able to make a base call for.

**Paired-end reads:**  
If you sequenced paired-end reads, you will have two FASTQ files:  
**..._R1.fastq** - contains the forward reads  
**..._R2.fastq**- contains the reverse reads  

Most downstream analysis tools will recognize that such files are paired-end, and the reads in the forward file correspond to the reads in the reverse file, although you often have to specify the names of both files to these tools.

It is critical that the R1 and R2 files have **the same number of records in both files**. If one has more records than the other, which can sometimes happen if there was an issue in the demultiplexing process, you will experience problems using these files as paired-end reads in downstream analyses.


Normally to check the quality of a fastQ file one would use software designed for that purpose, but it is useful to be able to manipulate FASTQ files if you are going to be doing lots of bioinformatics.

Due to their large size, we often perform `gzip` compression of FASTQ file. However this means we have to unzip them if we want to look inside them and perform operations on them. We can do this with the `zcat` command and a pipe (|).

In [ ]:
%%bash

# "zcat" lists the contents of a zipped file to your screen, and head limits the output to the first ten lines.

zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | head
zcat gcp_research_workflow/SRR1039508_2.chr20.fastq.gz | head

# Use the "wc -l" command to determine how many reads we have in total (don't forget to divide by 4 as each read has 4 lines affiliated with it)
# Paired-end reads should have the same number of records!

zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | wc -l
zcat gcp_research_workflow/SRR1039508_2.chr20.fastq.gz | wc -l


#### sed

Now let's determine the number of times the sequencer was not able to determine the base call for the first 10,000 reads. To do this we need to extract the sequence lines from the fastq entries, we will use the `sed` command to isolate the second line (sequence line) of each of the first 10,000 reads.

1. `sed`'s' `'p'` argument tells the program we want the output to be printed, and the `-n` option to tell sed we want to suppress automatic printing (so we don't get the results printed 2x). We specify `'2~4p'` as we want sed to *print the 2cnd line, then skip forward 4* to the next line of sequence. 
2. `grep` command with the `-o` flag to separate each letter (base pair) onto it's own line 
3. `sort` command sorts the letters alphabetically
4. `grep` again isolates lines with a base call of **N** 
5. `uniq -c` to count the number of times N was called in the first 10,000 reads. 

Whew .. it's a long command with a lot going on. I really encourage you to play with this command and see what the output looks as you add each successive pipe. 


In [ ]:
%%bash

# Step 1
# Print the first 10,000 reads
zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | sed -n '2~4p' | head -n 10000 

In [ ]:
%%bash

# Step 2
# Print each base from the first 10,000 reads to it's own line
zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | sed -n '2~4p' | head -n 10000 | grep -o . 

In [ ]:
%%bash

# Step 3
# Sort the bases from the first 10,000 reads
zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | sed -n '2~4p' | head -n 10000 | grep -o . | sort 

In [ ]:
%%bash

# Step 4
# Isolate the "N" bases from the first 10,000 reads
zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | sed -n '2~4p' | head -n 10000 | grep -o . | sort | grep 'N' 

In [ ]:
%%bash

# Step 5
# Count the number of "N" bases from the first 10,000 reads
zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | sed -n '2~4p' | head -n 10000 | grep -o . | sort | grep 'N' | uniq -c


## One of the maddening things about learning to code is that there are many ways to get to the same answer here are four other ways we could have gotten the same answer, test them out! 

# Count the number of "N" bases from the first 10,000 reads
# zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | sed -n '2~4p' | head -n 10000 | grep -o . | sort | grep 'N' | wc -l

# count the number of "N" bases from the first 10,000 reads
# zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | sed -n '2~4p' | head -n 10000 | grep -o . | sort | grep -c 'N'

# count the number of "N" bases from the first 10,000 reads
# zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | sed -n '2~4p' | head -n 10000 | grep -o . | grep 'N' | wc -l

# count the number of "N" bases from the first 10,000 reads
# zcat gcp_research_workflow/SRR1039508_1.chr20.fastq.gz | sed -n '2~4p' | head -n 10000 | grep -o . | grep -c 'N'


<div class="alert alert-block alert-warning">
    <i class="fa fa-question-circle-o" aria-hidden="true"></i>
    <b>TEST YOUR SKILLS</b> 
      <p>Practice your skills in the code block below</p>
    <div style="background-color: white ; color:black; padding: 3px;">Try editing the command above to check how many bases have a Q-score of 2 in the first 10,000 reads.<br><br> HINT: Use the table above to remind yourself of the ASCII symbol for Q-score =2 and remember the Q score is on the 4th line.<br><br> Run the #FLASHCARD code block to see the answer. </div>
    
</div>

In [ ]:
%%bash
## TEST YOUR SKILLS (enter and run your answers here)

# Try editing the command above to check how many bases have a Q-score of 0 in the first 10,000 reads

# Use the table above to remind yourself what the ASCII symbol for Q-score =2 is 

# Remember the Q score is on the fourth line not the second


In [ ]:
# FLASHCARD
from IPython.display import IFrame
IFrame("quiz_files/quiz3-2.html", width=600, height=250)